In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('max_rows', 20)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [3]:
AQI_FILEPATH = os.path.join(os.curdir, 'data', 'aqi.csv')

df = pd.read_csv(filepath_or_buffer=AQI_FILEPATH, parse_dates=['datetime'], index_col='datetime', na_values='NR')
df_bak = df.copy()

# 保留降雨量為 NR 值的資料
df_withnr = pd.read_csv(filepath_or_buffer=AQI_FILEPATH, parse_dates=['datetime'], index_col='datetime')
df_withnr_bak = df_withnr.copy()

# 保留還未將 datetime 設為 index 的資料
df_noindex = pd.read_csv(filepath_or_buffer=AQI_FILEPATH)
df_noindex_bak = df_noindex.copy()

# 讀取未處理過的原始資料
AQI_ORIG_FILEPATH = os.path.join(os.curdir, 'data', 'aqi_original.csv')

df_orig = pd.read_csv(AQI_ORIG_FILEPATH)
df_orig_bak = df_orig.copy()

# 讀取還未使用 pivot_table() 處理過的資料
AQI_NOPIVOT_FILEPATH = os.path.join(os.curdir, 'data', 'aqi_nopivot.csv')

df_nopivot = pd.read_csv(AQI_NOPIVOT_FILEPATH, na_values=['NR'])
df_nopivot.replace(r'[-]?\D*[.]?\D*[#*x]+', np.nan, regex=True, inplace=True)
df_nopivot.loc[:, 'value'] = df_nopivot.loc[:, 'value'].astype(np.float64)
df_nopivot_bak = df_orig.copy()

In [4]:
df.head()

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
datetime,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,20.0,2.0,0.20,0.04,1.7,7.5,9.2,25.0,67.0,18.0,NaN,88.0,1.5,2.0,33.0,37.0,1.4,0.1
2017-01-01 01:00:00,20.0,2.2,0.19,0.05,1.9,7.9,9.8,18.0,52.0,14.0,NaN,88.0,2.1,2.2,76.0,143.0,0.5,0.6
2017-01-01 02:00:00,19.0,2.2,0.24,0.08,2.0,9.3,11.0,13.0,59.0,17.0,NaN,89.0,2.0,2.3,140.0,142.0,0.5,0.6
2017-01-01 03:00:00,19.0,2.4,0.24,0.11,1.4,9.5,11.0,8.5,53.0,24.0,NaN,90.0,1.8,2.5,107.0,51.0,0.5,0.4
2017-01-01 04:00:00,18.0,3.4,0.23,0.12,6.8,11.0,17.0,1.8,37.0,25.0,NaN,90.0,1.7,3.6,96.0,106.0,1.0,0.5


# 第 7 章：結合資料

有時候要處理資料時，會拿到許多不同的檔案，這些檔案可能代表著整體現象的某一部份。這些資料可能是因為記錄方式的不同、來源位置不同、甚至有可能是擷取的方式不同，造成資料格式迥異、樣態差距也很大，也因此需要將這些資料結合，起來，再來製作分析。

## `concat()`：根據座標軸來結合資料

若資料集因為某些原因被分段，在後續使用時需要被重組起來的話，就會用到 `concat()` Function。舉例來說：若我們的空氣品質資料集被依月份分為十二等分，若運算時需要使用整年的資料，就要用 `concat()` Function 給結合起來再開始運算。

在使用時，依據所結合的資料的方向，可以使用 parameter `axis` 來變更，預設為 `0` 或 `index`。

如果結合的方向是 `0` 或 `index` 的話，還可以使用一個更簡易的 Function：`append()`。

* 常用 Parameters：
    * `join`：結合時遇到相同名稱的 column/row labels 時，參考的結合方法，預設為 `outer`
        * `outer`：將相同 column/row label 的資料結合，不相同的 label 則**另外存放**
        * `inner`：只將相同 column/row label 的資料結合，不相同的 label 則**捨棄**
    * `axis`：結合方向，預設為 `0`
        * `0`：Index (Row) 方向
        * `1`：Column 方向
* 參考文件：
    * [pandas.concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html)
    * [pandas.DataFrame.append](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html)
    * [pandas.Series.append](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.append.html)
    * [10 MInutes to Pandas | Merge | Concat](https://pandas.pydata.org/pandas-docs/stable/10min.html#concat)
    * [10 MInutes to Pandas | Merge | Append](https://pandas.pydata.org/pandas-docs/stable/10min.html#append)
    * [Merge, join, and concatenate | Concatenating objects](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects)
    * [Merge, join, and concatenate | Concatenating objects | Concatenating using append](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-using-append)

In [5]:
# 由資料集擷取出三部分的資料：2017-01-01 00:00:00 - 01:00:00、
# 2017-01-01 02:00:00 - 03:00:00、以及 2017-01-01 04:00:00 - 05:00:00
twelve_one_oclock = df.loc['2017-01-01 00:00:00':'2017-01-01 01:00:00', :]
two_three_oclock = df.loc['2017-01-01 02:00:00':'2017-01-01 03:00:00', :]
four_five_oclock = df.loc['2017-01-01 04:00:00':'2017-01-01 05:00:00', :]

In [6]:
# 依照 index (row) 方向結合三份資料
pd.concat([twelve_one_oclock,
           two_three_oclock,
           four_five_oclock], axis=0)
# 以上操作與以下等價
# twelve_one_oclock.append([two_three_oclock, four_five_oclock])

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
datetime,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,20.0,2.0,0.20,0.04,1.7,7.5,9.2,25.0,67.0,18.0,NaN,88.0,1.5,2.0,33.0,37.0,1.4,0.1
2017-01-01 01:00:00,20.0,2.2,0.19,0.05,1.9,7.9,9.8,18.0,52.0,14.0,NaN,88.0,2.1,2.2,76.0,143.0,0.5,0.6
2017-01-01 02:00:00,19.0,2.2,0.24,0.08,2.0,9.3,11.0,13.0,59.0,17.0,NaN,89.0,2.0,2.3,140.0,142.0,0.5,0.6
2017-01-01 03:00:00,19.0,2.4,0.24,0.11,1.4,9.5,11.0,8.5,53.0,24.0,NaN,90.0,1.8,2.5,107.0,51.0,0.5,0.4
2017-01-01 04:00:00,18.0,3.4,0.23,0.12,6.8,11.0,17.0,1.8,37.0,25.0,NaN,90.0,1.7,3.6,96.0,106.0,1.0,0.5
2017-01-01 05:00:00,17.0,3.6,0.37,0.13,8.4,12.0,20.0,2.0,57.0,28.0,NaN,91.0,2.0,3.7,79.0,114.0,0.8,0.7


In [7]:
# 由資料集擷取出三份資料，各為 5 rows, 6 columns
left_frame = df.iloc[:5, :6]
middle_frame = df.iloc[:5, 6:12]
right_frame = df.iloc[:5, 12:]

In [8]:
# 依照 column 方向結合三份資料，結合時會以相同 index 為結合的依據
pd.concat([left_frame, middle_frame, right_frame], axis=1)

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
datetime,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,20.0,2.0,0.20,0.04,1.7,7.5,9.2,25.0,67.0,18.0,NaN,88.0,1.5,2.0,33.0,37.0,1.4,0.1
2017-01-01 01:00:00,20.0,2.2,0.19,0.05,1.9,7.9,9.8,18.0,52.0,14.0,NaN,88.0,2.1,2.2,76.0,143.0,0.5,0.6
2017-01-01 02:00:00,19.0,2.2,0.24,0.08,2.0,9.3,11.0,13.0,59.0,17.0,NaN,89.0,2.0,2.3,140.0,142.0,0.5,0.6
2017-01-01 03:00:00,19.0,2.4,0.24,0.11,1.4,9.5,11.0,8.5,53.0,24.0,NaN,90.0,1.8,2.5,107.0,51.0,0.5,0.4
2017-01-01 04:00:00,18.0,3.4,0.23,0.12,6.8,11.0,17.0,1.8,37.0,25.0,NaN,90.0,1.7,3.6,96.0,106.0,1.0,0.5


## `merge()`：以 Column 數值相同為依據來結合資料

若多份資料中有重複的 Column，且可以用來當作結合資料的依據，用 `merge()` Function 來結合便是一種便利的方法。例如：我們有兩份資料，分別擁有相對濕度、風速（Column 名稱：`RH`, `WIND_SPEED`） 的資料，且兩份都擁有時間（Column 名稱：`datetime`）的資料，就可以用 `merge()` Function 將她們結合起來。

跟[關聯式資料庫](https://zh.wikipedia.org/zh-tw/關聯式資料庫)的 `JOIN` 操作相仿，JOIN 的時候可以選擇不同的 JOIN 方法：

* 僅保留兩側都有相同資料的 **INNER JOIN**
* 以一側為基準，指定保留該側所有資料的 **OUTER JOIN**：
    * 保留左側資料的 **LEFT OUTER JOIN**
    * 保留右側資料的 **RIGHT OUTER JOIN**

指定 JOIN 依據的 Column 是透過 parameter `on` 來指定，而選擇的 JOIN 類型是透過 parameter `how` 來設定。

* 常用 Parameters：
    * `how`：JOIN 的方法，預設為 `1nner`
        * `inner`： INNER JOIN
        * `left`： LEFT OUTER JOIN
        * `right`： RIGHT OUTER JOIN
        * `outer`： JULL OUTER JOIN
    * `on`：要用來比對的 label
* 參考文件：
    * [Merge, join, and concatenate | Database-style DataFrame joining/merging](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging)
    * [pandas.merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html)
    * [pandas.DataFrame.merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)
    * [10 Minutes to Pandas | Join](https://pandas.pydata.org/pandas-docs/stable/10min.html#join)
    * [關聯式資料庫 | Wikipedia](https://zh.wikipedia.org/zh-tw/關聯式資料庫)
    * [SQL 連接 | Wikipedia](https://zh.wikipedia.org/zh-tw/連接_%28SQL%29)

> [備註] 資料庫的 OUTER JOIN 其實還有一種  **FULL OUTER JOIN**，但實際的應用情境實在相當少見，所以暫不介紹

In [9]:
# 各自擁有相對濕度、風速（column label：'RH', 'WIND_SPEED'） 的資料
# 且兩份都擁有時間（column label：'datetime'）的資料
df_rh_left = df_noindex.loc[:4, ['datetime', 'RH']]  # 一份擷取 5 rows，置於左側

# 預覽資料
display(df_rh_left)

df_windspeed_right = df_noindex.loc[:9, 
                                 ['datetime',
                                  'WIND_SPEED']]  # 另一份擷取 10 rows，置於右側

# 預覽資料
display(df_windspeed_right)

,datetime,RH
0,2017-01-01 00:00:00,88.0
1,2017-01-01 01:00:00,88.0
2,2017-01-01 02:00:00,89.0
3,2017-01-01 03:00:00,90.0
4,2017-01-01 04:00:00,90.0


,datetime,WIND_SPEED
0,2017-01-01 00:00:00,1.4
1,2017-01-01 01:00:00,0.5
2,2017-01-01 02:00:00,0.5
3,2017-01-01 03:00:00,0.5
4,2017-01-01 04:00:00,1.0
5,2017-01-01 05:00:00,0.8
6,2017-01-01 06:00:00,1.1
7,2017-01-01 07:00:00,1.1
8,2017-01-01 08:00:00,1.5
9,2017-01-01 09:00:00,1.9


In [10]:
# INNER JOIN
# JOIN 時如果不自行指定 JOIN 依據的 column，Pandas 會依據輸入的資料
# 自動找尋所有名稱相同的 column 作為依據
pd.merge(left=df_rh_left, right=df_windspeed_right, how='inner')
# 此操作與以下等價
# pd.merge(left=df_rh, right=df_windspeed, how='inner', on=['inner'])

,datetime,RH,WIND_SPEED
0,2017-01-01 00:00:00,88.0,1.4
1,2017-01-01 01:00:00,88.0,0.5
2,2017-01-01 02:00:00,89.0,0.5
3,2017-01-01 03:00:00,90.0,0.5
4,2017-01-01 04:00:00,90.0,1.0


In [11]:
# LEFT OUTER JOIN
pd.merge(left=df_rh_left, right=df_windspeed_right,
         how='left', on=['datetime'])

,datetime,RH,WIND_SPEED
0,2017-01-01 00:00:00,88.0,1.4
1,2017-01-01 01:00:00,88.0,0.5
2,2017-01-01 02:00:00,89.0,0.5
3,2017-01-01 03:00:00,90.0,0.5
4,2017-01-01 04:00:00,90.0,1.0


In [12]:
# RIGHT OUTER JOIN
pd.merge(left=df_rh_left, right=df_windspeed_right,
         how='right', on=['datetime'])

,datetime,RH,WIND_SPEED
0,2017-01-01 00:00:00,88.0,1.4
1,2017-01-01 01:00:00,88.0,0.5
2,2017-01-01 02:00:00,89.0,0.5
3,2017-01-01 03:00:00,90.0,0.5
4,2017-01-01 04:00:00,90.0,1.0
5,2017-01-01 05:00:00,NaN,0.8
6,2017-01-01 06:00:00,NaN,1.1
7,2017-01-01 07:00:00,NaN,1.1
8,2017-01-01 08:00:00,NaN,1.5
9,2017-01-01 09:00:00,NaN,1.9
